In [ ]:
import ase.build
import ase.collections
import nglview
import ipywidgets as ipw

In [ ]:
def on_mol_change(c):
    global atoms
    atoms = ase.build.molecule(inp_mol.value)
    viewer.clear()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.center()

inp_mol = ipw.Dropdown(description="Molecule:", options=sorted(ase.collections.g2.names), value="H2O")
inp_mol.observe(on_mol_change, names='value')

viewer = nglview.NGLWidget()
on_mol_change(None)
display(inp_mol, viewer)

In [ ]:
def build_inp():
    inp = """
&GLOBAL
&END GLOBAL
&FORCE_EVAL
    METHOD Quickstep
    &DFT
       BASIS_SET_FILE_NAME BASIS_MOLOPT
       POTENTIAL_FILE_NAME GTH_POTENTIALS
       &XC
          &XC_FUNCTIONAL PBE
          &END XC_FUNCTIONAL
       &END XC
       &PRINT
          &MO_CUBES
             NHOMO 5
             NLUMO 5
          &END MO_CUBES
       &END PRINT
    &END DFT
    &SUBSYS
       &CELL
          ABC 5.0 5.0 5.0
       &END CELL
       &TOPOLOGY
          COORD_FILE_NAME coords.xyz
          COORD_FILE_FORMAT XYZ
       &END TOPOLOGY
       &KIND C
          BASIS_SET DZVP-MOLOPT-GTH
          POTENTIAL GTH-PBE
       &END KIND
       &KIND H
          BASIS_SET DZVP-MOLOPT-GTH
          POTENTIAL GTH-PBE
       &END KIND
       &KIND O
          BASIS_SET DZVP-MOLOPT-GTH
          POTENTIAL GTH-PBE
       &END KIND
    &END SUBSYS
&END FORCE_EVAL
"""
    return inp

In [ ]:
def run_cp2k(b):
    atoms.write("coords.xyz")
    open("demo.inp", "w").write(build_inp())
    ! cat demo.inp
    ! cp2k.popt demo.inp
    
btn_run = ipw.Button(description="Run CP2K")
btn_run.on_click(run_cp2k)
display(btn_run)